imports

In [1]:
import pandas as pd
import torch
import numpy as np
# import torchvision
import matplotlib.pyplot as plt
import time
import copy
from torch.utils.data import Dataset

In [2]:
# pd.read_csv("data_for_competition/test_target.csv")

In [12]:
file_names = [f'data_for_competition/train_data/train_data_{i}.pq' for i in range(12)]
# file_names

In [13]:
# s=0
# for i in range(12):
#     df = pd.read_parquet(f'data_for_competition/train_data/train_data_{i}.pq', engine='fastparquet')#[:100]
#     s+=df.shape[0]
#     print(df.groupby("id").rn.count().max())
# # print(s)

In [22]:
df = pd.read_parquet(f'data_for_competition/train_data/train_data_{0}.pq', engine='fastparquet')[:100]
df.id

0      0
1      0
2      0
3      0
4      0
      ..
95    11
96    11
97    12
98    12
99    12
Name: id, Length: 100, dtype: int64

In [15]:
a = np.array([[1.,  1.,  1.,  1.,  1.],
              [1.,  1.,  1.,  1.,  1.],
              [1.,  1.,  1.,  1.,  1.]])
np.pad(a, [(0, 10-a.shape[0]), (0, 0)], mode='constant')


array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [16]:
def get_padded_tensor(df, orders_cnt=70):
    tensors = []
    ids =[]
    for id, order in df.groupby("id"):
        order = order.sort_values("rn").drop('rn', axis=1).values
        tensors.append(torch.tensor(np.pad(order, [(0, orders_cnt-order.shape[0]), (0, 0)], mode='constant')))
        ids.append(id)
    return ids, torch.nn.utils.rnn.pad_sequence(tensors, batch_first=True)


In [17]:
# df[:30]

Loader

In [29]:
class CustomDataset(Dataset):
    def __init__(self, filenames):
      # `filenames` is a list of strings that contains all file names.
      # `batch_size` determines the number of files that we want to read in a chunk.
        self.filenames = filenames

    def __len__(self):
        # Number of chunks.
        return len(self.filenames)

    def __getitem__(self, idx):  # idx means index of the chunk.
      # In this method, we do all the preprocessing.
      # First read  data from files in a chunk. Preprocess it. Extract labels. Then return data and labels.
        # This extracts one batch of file names from the list `filenames`.
        file = self.filenames[idx]
        data = pd.read_parquet(file, engine='fastparquet')
        cat_columns = list(data.columns)[2:]
        data[cat_columns] = data[cat_columns].astype('category')

        # ids, data = get_padded_tensor(data)

        target_file_name = f'data_for_competition/train_target.csv'
        target = pd.read_csv(target_file_name, index_col='id')
        labels = target[target.index.isin(set(data.id))].flag.values
        # The following condition is actually needed in Pytorch. Otherwise, for our particular example, the iterator will be an infinite loop.
        # Readers can verify this by removing this condition.
        if idx == self.__len__():
            raise IndexError

        return data, labels


In [30]:
ds = CustomDataset(file_names)

In [31]:
for data, labels in ds:
    print(data.shape)
    # print(labels.shape)
    break

(1974724, 61)


In [7]:
data

array([[[     0,      1,     18, ...,      1,      0,      0],
        [     0,      2,     18, ...,      1,      0,      0],
        [     0,      3,     18, ...,      1,      1,      1],
        ...,
        [249999,      1,      9, ...,      1,      0,      1],
        [249999,      2,      9, ...,      1,      0,      0],
        [249999,      3,      9, ...,      1,      0,      0]]])

In [ ]:
def train_model(model, loss, optimizer, scheduler, num_epochs):
    for epoch in range(num_epochs):
        print('Epoch {}/{}:'.format(epoch, num_epochs - 1), flush=True)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                dataloader = train_dataloader
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                dataloader = val_dataloader
                model.eval()   # Set model to evaluate mode

            running_loss = 0.
            running_acc = 0.

            # Iterate over data.
            for inputs, labels in tqdm(dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # forward and backward
                with torch.set_grad_enabled(phase == 'train'):
                    preds = model(inputs)
                    loss_value = loss(preds, labels)
                    preds_class = preds.argmax(dim=1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss_value.backward()
                        optimizer.step()

                # statistics
                running_loss += loss_value.item()
                running_acc += (preds_class == labels.data).float().mean()

            epoch_loss = running_loss / len(dataloader)
            epoch_acc = running_acc / len(dataloader)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc), flush=True)

    return model